# FLDetector for CIFAR10 with Fang/Dirichlet distribution


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/scratch/gypsum-gpu105/4483553/ipykernel_2188274/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [1]:
import collections
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import random
import copy
import time
from functools import reduce
from torchsummary import summary

import os
import sys
import pickle
sys.path.insert(0,'./utils/')
from logger import *
from eval import *
from misc import *

from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from collections import defaultdict

from SGD import *
import copy
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [2]:
from cifar10_models import *
from cifar10_util import *

In [3]:
def sample_dirichlet_train_data(trainset, no_participants, alpha=0.9, force=False):
        """
            Input: Number of participants and alpha (param for distribution)
            Output: A list of indices denoting data in CIFAR training set.
            Requires: cifar_classes, a preprocessed class-indice dictionary.
            Sample Method: take a uniformly sampled 10-dimension vector as parameters for
            dirichlet distribution to sample number of images in each class.
        """
        if not os.path.exists('./dirichlet_a_%.1f_nusers_%d.pkl'%(alpha, no_participants)) or force:
            print('generating participant indices for alpha %.1f'%alpha)
            np.random.seed(0)
            cifar_classes = {}
            for ind, x in enumerate(trainset):
                _, label = x
                if label in cifar_classes:
                    cifar_classes[label].append(ind)
                else:
                    cifar_classes[label] = [ind]

            per_participant_list = defaultdict(list)
            no_classes = len(cifar_classes.keys())
            for n in range(no_classes):
                random.shuffle(cifar_classes[n])
                sampled_probabilities = len(cifar_classes[n]) * np.random.dirichlet(
                    np.array(no_participants * [alpha]))
                for user in range(no_participants):
                    no_imgs = int(round(sampled_probabilities[user]))
                    sampled_list = cifar_classes[n][:min(len(cifar_classes[n]), no_imgs)]
                    per_participant_list[user].extend(sampled_list)
                    cifar_classes[n] = cifar_classes[n][min(len(cifar_classes[n]), no_imgs):]
            with open('./dirichlet_a_%.1f_nusers_%d.pkl'%(alpha, no_participants), 'wb') as f:
                pickle.dump(per_participant_list, f)
        else:
            per_participant_list = pickle.load(open('./dirichlet_a_%.1f_nusers_%d.pkl'%(alpha, no_participants), 'rb'))

        return per_participant_list

In [4]:
def get_fang_train_data(trainset, num_workers=100, bias=0.5, force=False):
    dist_file = 'fang_nworkers%d_bias%.1f.pkl' % (num_workers, bias)
    if not force and os.path.exists(dist_file):
        print('Loading fang distribution for num_workers %d and bias %.1f from memory' % (num_workers, bias))
        return pickle.load(open(dist_file, 'rb'))
    bias_weight = bias
    other_group_size = (1 - bias_weight) / 9.
    worker_per_group = num_workers / 10
    each_worker_data = [[] for _ in range(num_workers)]
    each_worker_label = [[] for _ in range(num_workers)]
    per_participant_list = defaultdict(list)
    for i, (x, y) in enumerate(trainset):
        # assign a data point to a group
        upper_bound = (y) * (1 - bias_weight) / 9. + bias_weight
        lower_bound = (y) * (1 - bias_weight) / 9.
        rd = np.random.random_sample()
        if rd > upper_bound:
            worker_group = int(np.floor((rd - upper_bound) / other_group_size) + y + 1)
        elif rd < lower_bound:
            worker_group = int(np.floor(rd / other_group_size))
        else:
            worker_group = y
        rd = np.random.random_sample()
        selected_worker = int(worker_group * worker_per_group + int(np.floor(rd * worker_per_group)))
        per_participant_list[selected_worker].extend([i])
    
    print('Saving fang distribution for num_workers %d and bias %.1f to memory' % (num_workers, bias))
    pickle.dump(per_participant_list, open(dist_file, 'wb'))
    return per_participant_list

In [5]:
def get_federated_data(trainset, num_workers, distribution='fang', param=1, force=False):
    if distribution == 'fang':
        per_participant_list = get_fang_train_data(trainset, num_workers, bias=param, force=force)
    elif distribution == 'dirichlet':
        per_participant_list = sample_dirichlet_train_data(trainset, num_workers, alpha=param, force=force)

    each_worker_idx = [[] for _ in range(num_workers)] 
    each_worker_te_idx = [[] for _ in range(num_workers)]
    np.random.seed(0)
    for worker_idx in range(len(per_participant_list)):
        w_indices = np.array(per_participant_list[worker_idx])
        w_len = len(w_indices)
        len_tr = int(5*w_len/6)
        tr_idx = np.random.choice(w_len, len_tr, replace=False)
        te_idx = np.delete(np.arange(w_len), tr_idx)
        each_worker_idx[worker_idx] = w_indices[tr_idx]
        each_worker_te_idx[worker_idx] = w_indices[te_idx]
    global_test_idx = np.concatenate(each_worker_te_idx)
    return each_worker_idx, each_worker_te_idx, global_test_idx

In [6]:
def get_train(dataset, indices, batch_size=32, shuffle=False):
    train_loader = torch.utils.data.DataLoader(dataset,
                                               batch_size=batch_size,
                                               sampler=torch.utils.data.sampler.SubsetRandomSampler(indices))
    
    return train_loader

In [7]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
data_loc='/home/vshejwalkar_umass_edu/data/'
# load the train dataset

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

cifar10_train = datasets.CIFAR10(root=data_loc, train=True, download=True, transform=transform_train)
cifar10_test = datasets.CIFAR10(root=data_loc, train=False, download=True, transform=transform_train)

te_cifar10_train = datasets.CIFAR10(root=data_loc, train=True, download=True, transform=transform_test)
te_cifar10_test = datasets.CIFAR10(root=data_loc, train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [8]:
def full_trim(v, f):
    '''
    Full-knowledge Trim attack. w.l.o.g., we assume the first f worker devices are compromised.
    v: the list of squeezed gradients
    f: the number of compromised worker devices
    '''
    vi_shape = v[0].unsqueeze(0).T.shape
    v_tran = v.T
    
    maximum_dim = torch.max(v_tran, dim=1)
    maximum_dim = maximum_dim[0].reshape(vi_shape)
    minimum_dim = torch.min(v_tran, dim=1)
    minimum_dim = minimum_dim[0].reshape(vi_shape)
    direction = torch.sign(torch.sum(v_tran, dim=-1, keepdims=True))
    directed_dim = (direction > 0) * minimum_dim + (direction < 0) * maximum_dim

    for i in range(f):
        random_12 = 2
        tmp = directed_dim * ((direction * directed_dim > 0) / random_12 + (direction * directed_dim < 0) * random_12)
        tmp = tmp.squeeze()
        v[i] = tmp
    return v

In [9]:
def tr_mean(all_updates, n_attackers):
    sorted_updates = torch.sort(all_updates, 0)[0]
    out = torch.mean(sorted_updates[n_attackers:-n_attackers], 0) if n_attackers else torch.mean(sorted_updates,0)
    return out

In [10]:
from torch.nn.utils import parameters_to_vector, vector_to_parameters

def train(trainloader, model, model_received, criterion, optimizer, pgd=False, eps=2):
    model.train()
    losses = AverageMeter()
    top1 = AverageMeter()
    for batch_ind, (inputs, targets) in enumerate(trainloader):
        inputs = inputs.to(device, torch.float)
        targets = targets.to(device, torch.long)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.item(), inputs.size()[0])
        top1.update(prec1.item()/100.0, inputs.size()[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if pgd:
            curr_model = list(model.parameters())
            curr_model_vec = parameters_to_vector(curr_model)
            if torch.norm(curr_model_vec - model_received) > eps:
                curr_model_vec = eps*(curr_model_vec - model_received)/torch.norm(curr_model_vec - model_received) + model_received
                vector_to_parameters(curr_model_vec, curr_model)
        
    return (losses.avg, top1.avg)

def test(testloader, model, criterion):
    model.eval()
    losses = AverageMeter()
    top1 = AverageMeter()
    for batch_ind, (inputs, targets) in enumerate(testloader):
        inputs = inputs.to(device, torch.float)
        targets = targets.to(device, torch.long)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        top1.update(prec1/100.0, inputs.size()[0])
    return (losses.avg, top1.avg)

# resnet-BN

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.autograd import Variable

__all__ = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110', 'resnet1202']

def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def resnet20():
    return ResNet(BasicBlock, [3, 3, 3])

# FLDetector utils

In [12]:
from sklearn.metrics import roc_auc_score
def fldetector(old_gradients, user_grads, b=0, hvp=None, agr='tr_mean'):
    if hvp is not None:
        hvp = torch.from_numpy(hvp).to(device)
        pred_grad = copy.deepcopy(old_gradients)
        distance = []
        for i in range(len(old_gradients)):
            pred_grad[i] += hvp
        pred = np.zeros(100)
        pred[:b] = 1
        distance = torch.norm(pred_grad - user_grads, dim = 1).cpu().numpy()
        distance = distance / np.sum(distance)
    else:
        distance = None
    
    if agr == 'average':
        agg_grads = torch.mean(user_grads, 0)
    elif agr == 'median':
        agg_grads = torch.median(user_grads, 0)[0]
    elif agr == 'tr_mean':
        agg_grads = tr_mean(user_grads, b)
    return agg_grads, distance

def lbfgs(S_k_list, Y_k_list, v):
    curr_S_k = torch.stack(S_k_list).T
    curr_Y_k = torch.stack(Y_k_list).T
    S_k_time_Y_k = np.dot(curr_S_k.T.cpu().numpy(), curr_Y_k.cpu().numpy())
    S_k_time_S_k = np.dot(curr_S_k.T.cpu().numpy(), curr_S_k.cpu().numpy())
    R_k = np.triu(S_k_time_Y_k)
    L_k = S_k_time_Y_k - R_k
    sigma_k = np.dot(Y_k_list[-1].unsqueeze(0).cpu().numpy(), S_k_list[-1].unsqueeze(0).T.cpu().numpy()) / (np.dot(S_k_list[-1].unsqueeze(0).cpu().numpy(), S_k_list[-1].unsqueeze(0).T.cpu().numpy()))
    D_k_diag = np.diag(S_k_time_Y_k)
    upper_mat = np.concatenate((sigma_k * S_k_time_S_k, L_k), axis=1)
    lower_mat = np.concatenate((L_k.T, -np.diag(D_k_diag)), axis=1)
    mat = np.concatenate((upper_mat, lower_mat), axis=0)
    mat_inv = np.linalg.inv(mat)

    approx_prod = sigma_k * v.cpu().numpy()
    approx_prod = approx_prod.T
    p_mat = np.concatenate((np.dot(curr_S_k.T.cpu().numpy(), sigma_k * v.unsqueeze(0).T.cpu().numpy()), np.dot(curr_Y_k.T.cpu().numpy(), v.unsqueeze(0).T.cpu().numpy())), axis=0)
    approx_prod -= np.dot(np.dot(np.concatenate((sigma_k * curr_S_k.cpu().numpy(), curr_Y_k.cpu().numpy()), axis=1), mat_inv), p_mat)

    return approx_prod

def detection(score, nobyz, nworkers):
    estimator = KMeans(n_clusters=2)
    estimator.fit(score.reshape(-1, 1))
    label_pred = estimator.labels_
    if np.mean(score[label_pred==0])<np.mean(score[label_pred==1]):
        #0 is the label of malicious clients
        label_pred = 1 - label_pred
    real_label=np.ones(nworkers)
    real_label[:nobyz]=0
    acc=len(label_pred[label_pred==real_label])/nworkers
    recall=1-np.sum(label_pred[:nobyz])/nobyz
    fpr=1-np.sum(label_pred[nobyz:])/(nworkers-nobyz)
    fnr=np.sum(label_pred[:nobyz])/nobyz
    auc = roc_auc_score(real_label, label_pred)
    print("acc %0.4f; recall %0.4f; fpr %0.4f; fnr %0.4f; auc %.4f" % (acc, recall, fpr, fnr, auc))
    return acc, fpr, fnr, auc
    # print(silhouette_score(score.reshape(-1, 1), label_pred))

def detection1(score, nobyz):
    nrefs = 10
    ks = range(1, 8)
    gaps = np.zeros(len(ks))
    gapDiff = np.zeros(len(ks) - 1)
    sdk = np.zeros(len(ks))
    min = np.min(score)
    max = np.max(score)
    score = (score - min)/(max-min)
    for i, k in enumerate(ks):
        estimator = KMeans(n_clusters=k)
        estimator.fit(score.reshape(-1, 1))
        label_pred = estimator.labels_
        center = estimator.cluster_centers_
        Wk = np.sum([np.square(score[m]-center[label_pred[m]]) for m in range(len(score))])
        WkRef = np.zeros(nrefs)
        for j in range(nrefs):
            rand = np.random.uniform(0, 1, len(score))
            estimator = KMeans(n_clusters=k)
            estimator.fit(rand.reshape(-1, 1))
            label_pred = estimator.labels_
            center = estimator.cluster_centers_
            WkRef[j] = np.sum([np.square(rand[m]-center[label_pred[m]]) for m in range(len(rand))])
        gaps[i] = np.log(np.mean(WkRef)) - np.log(Wk)
        sdk[i] = np.sqrt((1.0 + nrefs) / nrefs) * np.std(np.log(WkRef))

        if i > 0:
            gapDiff[i - 1] = gaps[i - 1] - gaps[i] + sdk[i]
    #print(gapDiff)
    for i in range(len(gapDiff)):
        if gapDiff[i] >= 0:
            select_k = i+1
            break
    if select_k == 1:
        return 0
    else:
        # print('Attack Detected!')
        return 1

In [14]:
all_data = torch.utils.data.ConcatDataset((cifar10_train, cifar10_test))
all_test_data = torch.utils.data.ConcatDataset((te_cifar10_train, te_cifar10_test))
batch_size = 16
num_workers = 100
distribution='fang'
param = .5
force = False

each_worker_idx, each_worker_te_idx, global_test_idx = get_federated_data(
    all_data, num_workers=100, distribution=distribution, param=param, force=force)

train_loaders = []
for pos, indices in enumerate(each_worker_idx):
    train_loaders.append((pos, get_train(all_data, indices, batch_size)))
test_loaders = []
for pos, indices in enumerate(each_worker_te_idx):
    test_loaders.append((pos, get_train(all_test_data, indices, len(indices))))
cifar10_test_loader = get_train(all_test_data, global_test_idx)

len(train_loaders), len(test_loaders), len(global_test_idx)

Loading fang distribution for num_workers 100 and bias 0.5 from memory


(100, 100, 10038)

In [15]:
'''
FAST BASELINE
'''

use_cuda = torch.cuda.is_available()
criterion = nn.CrossEntropyLoss()
n_malicious = [10]
n_runs = 3
results={}
for n_mal in n_malicious:
    nbyz = n_mal
    results[n_mal] = {'acc': [], 'fpr': [], 'fnr': [], 'auc': [], 'stop_e': []}
    for run in range(n_runs):
        torch.cuda.empty_cache()
        nepochs=200
        local_epochs = 2
        batch_size = 16
        num_workers = 100
        local_lr = 0.1
        global_lr = 1
        agr = 'tr_mean'
        best_global_acc=0
        epoch_num = 0

        fed_model = resnet20().cuda()
        model_received = []
        for i, (name, param) in enumerate(fed_model.state_dict().items()):
            model_received = param.view(-1).data.type(torch.cuda.FloatTensor) if len(model_received) == 0 else torch.cat((model_received, param.view(-1).data.type(torch.cuda.FloatTensor)))

        best_accs_per_round = []
        accs_per_round = []
        loss_per_round = []
        home_dir = '/home/vshejwalkar_umass_edu/fedrecover/'

        # FLDetector initializations
        weight_record = []
        grad_record = []
        test_grads = []
        old_grad_list = []
        malicious_scores = np.zeros((1, num_workers-(20-nbyz)))
        start_detection_epoch = 10
        window_size = 10
        assert (start_detection_epoch - window_size >= 0), 'start_detection_epoch %d should be more than window_size %d' % (start_detection_epoch, window_size)

        # Adaptive attack initializers
        good_distance_range = np.zeros((1, nbyz))
        attack_type = 'NDSS21'
        dev_type = 'std'
        
        while epoch_num <= nepochs:
            torch.cuda.empty_cache()
            round_clients = np.arange(20-nbyz, num_workers)
            round_benign = round_clients
            user_grads=[]
            benign_norm = 0
            for i in round_benign:
                model = copy.deepcopy(fed_model)
                optimizer = optim.SGD(model.parameters(), lr = local_lr*(0.999**epoch_num), momentum=0.9, weight_decay=1e-4)

                for epoch in range(local_epochs):
                    train_loss, train_acc = train(train_loaders[i][1], model, model_received, criterion, optimizer)

                params = []
                for i, (name, param) in enumerate(model.state_dict().items()):
                    params = param.view(-1).data.type(torch.cuda.FloatTensor) if len(params) == 0 else torch.cat((params, param.view(-1).data.type(torch.cuda.FloatTensor)))

                update =  (params - model_received)
                benign_norm += torch.norm(update)/len(round_benign)
                user_grads = update[None,:] if len(user_grads) == 0 else torch.cat((user_grads, update[None,:]), 0)

            weight = model_received

            if (epoch_num > start_detection_epoch):
                hvp = lbfgs(weight_record, grad_record, weight - last_weight)
                hvp = np.squeeze(hvp)
            else:
                hvp = None

            good_current_grads = copy.deepcopy(user_grads[:nbyz])
            if hvp is not None:
                pred_grad = copy.deepcopy(good_old_grads)
                distance = []
                for i in range(len(good_old_grads)):
                    pred_grad[i] += torch.from_numpy(hvp).to(device)
                good_distance_range = np.concatenate(
                    (good_distance_range, torch.norm(pred_grad - good_current_grads, dim = 1).cpu().numpy()[None,:]), 0)

            if attack_type != 'none' and (epoch_num < start_detection_epoch):
                user_grads = full_trim(user_grads, nbyz)
                # user_grads[:nbyz] = full_trim(user_grads[:nbyz], nbyz)
            elif epoch_num > start_detection_epoch:
                if attack_type == 'full_trim':
                    user_grads = full_trim(user_grads, nbyz)
                elif attack_type == 'none':
                    pass
                else:
                    if attack_type == 'NDSS21':
                        distance_bound = np.random.choice(np.mean(good_distance_range[-1:], 0))
                        model_re = torch.mean(good_current_grads, dim=0)
                        if dev_type == 'unit_vec':
                            deviation = model_re / torch.norm(model_re)  # unit vector, dir opp to good dir
                        elif dev_type == 'sign':
                            deviation = torch.sign(model_re)
                        elif dev_type == 'std':
                            deviation = torch.std(good_current_grads, 0)
                        # noise = deviation * ((distance_bound + np.random.uniform(0, np.std(good_distance_range[-1]))) / torch.norm(deviation))
                        noise = deviation * ((distance_bound)) / torch.norm(deviation)
                    elif attack_type == 'mod_trim':
                        mal_grads= full_trim(user_grads[:nbyz], nbyz)
                        pass
                    else:
                        noise = torch.zeros(hvp.shape).to(device)
                    for m in range(nbyz):
                        user_grads[m] = old_grad_list[m] + torch.from_numpy(hvp).to(device) + noise

            agg_grads, distance = fldetector(old_grad_list, user_grads, nbyz, hvp, agr=agr)

            if distance is not None and epoch_num > (start_detection_epoch - window_size):
                malicious_scores = np.concatenate((malicious_scores, distance[None, :]), 0)

            if malicious_scores.shape[0] >= window_size+1:
                if detection1(np.sum(malicious_scores[-window_size:], axis=0), nbyz):
                    print('!!!! Stop at iteration:', epoch_num)
                    print('r %d nmal %d | e %d benign_norm %.3f val loss %.3f val acc %.3f best val_acc %.3f'% (run, nbyz, epoch_num, benign_norm, val_loss, val_acc, best_global_acc))
                    acc, fpr, fnr, auc = detection(
                        np.sum(malicious_scores[-window_size:], axis=0), nbyz, len(user_grads))
                    results[n_mal]['acc'].append(acc)
                    results[n_mal]['fpr'].append(fpr)
                    results[n_mal]['fnr'].append(fnr)
                    results[n_mal]['auc'].append(auc)
                    results[n_mal]['stop_e'].append(epoch_num)
                    break
                    break

            if epoch_num > (start_detection_epoch - window_size):
                weight_record.append(weight - last_weight)
                grad_record.append(agg_grads - last_grad)

            if (len(weight_record) > window_size):
                del weight_record[0]
                del grad_record[0]

            last_weight = weight
            last_grad = agg_grads
            old_grad_list = user_grads
            good_old_grads = good_current_grads
            del user_grads

            model_received = model_received + global_lr * agg_grads
            fed_model = resnet20().cuda()
            start_idx=0
            state_dict = {}
            previous_name = 'none'
            for i, (name, param) in enumerate(fed_model.state_dict().items()):
                start_idx = 0 if i == 0 else start_idx + len(fed_model.state_dict()[previous_name].data.view(-1))
                start_end = start_idx + len(fed_model.state_dict()[name].data.view(-1))
                params = model_received[start_idx:start_end].reshape(fed_model.state_dict()[name].data.shape)
                state_dict[name] = params
                previous_name = name
            fed_model.load_state_dict(state_dict)
            val_loss, val_acc = test(cifar10_test_loader, fed_model, criterion)
            is_best = best_global_acc < val_acc
            best_global_acc = max(best_global_acc, val_acc)

            best_accs_per_round.append(best_global_acc.cpu().item())
            accs_per_round.append(val_acc.cpu().item())
            loss_per_round.append(val_loss.cpu().item())

            if epoch_num%5==0 or epoch_num==nepochs-1:
                print('r %d nmal %d | e %d benign_norm %.3f val loss %.3f val acc %.3f best val_acc %.3f'% (run, nbyz, epoch_num, benign_norm, val_loss, val_acc, best_global_acc))
            if math.isnan(val_loss) or val_loss > 100000:
                print('val loss %f... exit'%val_loss)
                break
            epoch_num+=1

        final_accs_per_client=[]
        for i in range(num_workers):
            client_loss, client_acc = test(test_loaders[i][1], fed_model, criterion)
            final_accs_per_client.append(client_acc.cpu().item())
        all_results = collections.OrderedDict(
            final_accs_per_client=np.array(final_accs_per_client),
            accs_per_round=np.array(accs_per_round),
            best_accs_per_round=np.array(best_accs_per_round),
            loss_per_round=np.array(loss_per_round),
            results=results
        )
        pickle.dump(all_results, open(os.path.join(home_dir, 'FLDetector_plots_data/rq4_adaptive_cifar10_fast_trmean_NDSS21_std_m%d_r%d.pkl' % (nbyz, run)), 'wb'))

r 0 nmal 10 | e 0 benign_norm 2695.255 val loss 2.308 val acc 0.099 best val_acc 0.099
r 0 nmal 10 | e 5 benign_norm 276.572 val loss 2.110 val acc 0.179 best val_acc 0.179
r 0 nmal 10 | e 10 benign_norm 274.452 val loss 1.727 val acc 0.364 best val_acc 0.364
r 0 nmal 10 | e 15 benign_norm 274.350 val loss 2.034 val acc 0.348 best val_acc 0.399
r 0 nmal 10 | e 20 benign_norm 274.948 val loss 1.964 val acc 0.380 best val_acc 0.399
!!!! Stop at iteration: 22
r 0 nmal 10 | e 22 benign_norm 275.277 val loss 2.013 val acc 0.380 best val_acc 0.399
acc 0.6778; recall 1.0000; fpr 0.3625; fnr 0.0000; auc 0.8187
r 1 nmal 10 | e 0 benign_norm 7436.722 val loss 2.316 val acc 0.099 best val_acc 0.099
r 1 nmal 10 | e 5 benign_norm 277.830 val loss 2.228 val acc 0.156 best val_acc 0.156
r 1 nmal 10 | e 10 benign_norm 275.389 val loss 1.747 val acc 0.353 best val_acc 0.353
r 1 nmal 10 | e 15 benign_norm 274.175 val loss 1.632 val acc 0.397 best val_acc 0.411
r 1 nmal 10 | e 20 benign_norm 275.201 val 

In [15]:
'''
FAST BASELINE
'''

use_cuda = torch.cuda.is_available()
criterion = nn.CrossEntropyLoss()
n_malicious = [10]
n_runs = 1
results={}
for n_mal in n_malicious:
    nbyz = n_mal
    results[n_mal] = {'acc': [], 'fpr': [], 'fnr': [], 'auc': [], 'stop_e': []}
    for run in range(n_runs):
        torch.cuda.empty_cache()
        nepochs=200
        local_epochs = 2
        batch_size = 16
        num_workers = 100
        local_lr = 0.1
        global_lr = 1
        agr = 'tr_mean'
        best_global_acc=0
        epoch_num = 0

        fed_model = resnet20().cuda()
        model_received = []
        for i, (name, param) in enumerate(fed_model.state_dict().items()):
            model_received = param.view(-1).data.type(torch.cuda.FloatTensor) if len(model_received) == 0 else torch.cat((model_received, param.view(-1).data.type(torch.cuda.FloatTensor)))

        best_accs_per_round = []
        accs_per_round = []
        loss_per_round = []
        home_dir = '/home/vshejwalkar_umass_edu/fedrecover/'

        # FLDetector initializations
        weight_record = []
        grad_record = []
        test_grads = []
        old_grad_list = []
        malicious_scores = np.zeros((1, num_workers-(20-nbyz)))
        start_detection_epoch = 10
        window_size = 10
        assert (start_detection_epoch - window_size >= 0), 'start_detection_epoch %d should be more than window_size %d' % (start_detection_epoch, window_size)

        # Adaptive attack initializers
        good_distance_range = np.zeros((1, nbyz))
        attack_type = 'NDSS21'
        dev_type = 'std'
        
        while epoch_num <= nepochs:
            torch.cuda.empty_cache()
            round_clients = np.arange(20-nbyz, num_workers)
            round_benign = round_clients
            user_grads=[]
            benign_norm = 0
            for i in round_benign:
                model = copy.deepcopy(fed_model)
                optimizer = optim.SGD(model.parameters(), lr = local_lr*(0.999**epoch_num), momentum=0.9, weight_decay=1e-4)

                for epoch in range(local_epochs):
                    train_loss, train_acc = train(train_loaders[i][1], model, model_received, criterion, optimizer)

                params = []
                for i, (name, param) in enumerate(model.state_dict().items()):
                    params = param.view(-1).data.type(torch.cuda.FloatTensor) if len(params) == 0 else torch.cat((params, param.view(-1).data.type(torch.cuda.FloatTensor)))

                update =  (params - model_received)
                benign_norm += torch.norm(update)/len(round_benign)
                user_grads = update[None,:] if len(user_grads) == 0 else torch.cat((user_grads, update[None,:]), 0)

            weight = model_received

            if (epoch_num > start_detection_epoch):
                hvp = lbfgs(weight_record, grad_record, weight - last_weight)
                hvp = np.squeeze(hvp)
            else:
                hvp = None

            good_current_grads = copy.deepcopy(user_grads[:nbyz])
            if hvp is not None:
                pred_grad = copy.deepcopy(good_old_grads)
                distance = []
                for i in range(len(good_old_grads)):
                    pred_grad[i] += torch.from_numpy(hvp).to(device)
                good_distance_range = np.concatenate(
                    (good_distance_range, torch.norm(pred_grad - good_current_grads, dim = 1).cpu().numpy()[None,:]), 0)

            if attack_type != 'none' and (epoch_num < start_detection_epoch):
                user_grads = full_trim(user_grads, nbyz)
                # user_grads[:nbyz] = full_trim(user_grads[:nbyz], nbyz)
            elif epoch_num > start_detection_epoch:
                if attack_type == 'full_trim':
                    user_grads = full_trim(user_grads, nbyz)
                elif attack_type == 'none':
                    pass
                else:
                    if attack_type == 'NDSS21':
                        distance_bound = np.mean(np.mean(good_distance_range[-1:], 0))
                        model_re = torch.mean(good_current_grads, dim=0)
                        if dev_type == 'unit_vec':
                            deviation = model_re / torch.norm(model_re)  # unit vector, dir opp to good dir
                        elif dev_type == 'sign':
                            deviation = torch.sign(model_re)
                        elif dev_type == 'std':
                            deviation = torch.std(good_current_grads, 0)
                        # noise = deviation * ((distance_bound + np.random.uniform(0, np.std(good_distance_range[-1]))) / torch.norm(deviation))
                        noise = deviation * ((distance_bound)) / torch.norm(deviation)
                    elif attack_type == 'mod_trim':
                        mal_grads= full_trim(user_grads[:nbyz], nbyz)
                        pass
                    else:
                        noise = torch.zeros(hvp.shape).to(device)
                    for m in range(nbyz):
                        user_grads[m] = old_grad_list[m] + torch.from_numpy(hvp).to(device) + noise

            agg_grads, distance = fldetector(old_grad_list, user_grads, nbyz, hvp, agr=agr)

            if distance is not None and epoch_num > (start_detection_epoch - window_size):
                malicious_scores = np.concatenate((malicious_scores, distance[None, :]), 0)

            if malicious_scores.shape[0] >= window_size+1:
                if detection1(np.sum(malicious_scores[-window_size:], axis=0), nbyz):
                    print('!!!! Stop at iteration:', epoch_num)
                    print('r %d nmal %d | e %d benign_norm %.3f val loss %.3f val acc %.3f best val_acc %.3f'% (run, nbyz, epoch_num, benign_norm, val_loss, val_acc, best_global_acc))
                    acc, fpr, fnr, auc = detection(
                        np.sum(malicious_scores[-window_size:], axis=0), nbyz, len(user_grads))
                    results[n_mal]['acc'].append(acc)
                    results[n_mal]['fpr'].append(fpr)
                    results[n_mal]['fnr'].append(fnr)
                    results[n_mal]['auc'].append(auc)
                    results[n_mal]['stop_e'].append(epoch_num)
                    break
                    break

            if epoch_num > (start_detection_epoch - window_size):
                weight_record.append(weight - last_weight)
                grad_record.append(agg_grads - last_grad)

            if (len(weight_record) > window_size):
                del weight_record[0]
                del grad_record[0]

            last_weight = weight
            last_grad = agg_grads
            old_grad_list = user_grads
            good_old_grads = good_current_grads
            del user_grads

            model_received = model_received + global_lr * agg_grads
            fed_model = resnet20().cuda()
            start_idx=0
            state_dict = {}
            previous_name = 'none'
            for i, (name, param) in enumerate(fed_model.state_dict().items()):
                start_idx = 0 if i == 0 else start_idx + len(fed_model.state_dict()[previous_name].data.view(-1))
                start_end = start_idx + len(fed_model.state_dict()[name].data.view(-1))
                params = model_received[start_idx:start_end].reshape(fed_model.state_dict()[name].data.shape)
                state_dict[name] = params
                previous_name = name
            fed_model.load_state_dict(state_dict)
            val_loss, val_acc = test(cifar10_test_loader, fed_model, criterion)
            is_best = best_global_acc < val_acc
            best_global_acc = max(best_global_acc, val_acc)
            if is_best:
                best_model = copy.deepcopy(fed_model)

            best_accs_per_round.append(best_global_acc.cpu().item())
            accs_per_round.append(val_acc.cpu().item())
            loss_per_round.append(val_loss.cpu().item())

            if epoch_num%5==0 or epoch_num==nepochs-1:
                print('r %d nmal %d | e %d benign_norm %.3f val loss %.3f val acc %.3f best val_acc %.3f'% (run, nbyz, epoch_num, benign_norm, val_loss, val_acc, best_global_acc))
            if math.isnan(val_loss) or val_loss > 100000:
                print('val loss %f... exit'%val_loss)
                break
            epoch_num+=1

        final_accs_per_client=[]
        for i in range(num_workers):
            client_loss, client_acc = test(test_loaders[i][1], best_model, criterion)
            final_accs_per_client.append(client_acc.cpu().item())
        all_results = collections.OrderedDict(
            final_accs_per_client=np.array(final_accs_per_client),
            accs_per_round=np.array(accs_per_round),
            best_accs_per_round=np.array(best_accs_per_round),
            loss_per_round=np.array(loss_per_round),
            results=results
        )
        pickle.dump(all_results, open(os.path.join(home_dir, 'FLDetector_plots_data/rq4_adaptive_cifar10_fast_trmean_NDSS21_std_m%d_r%d_personalized.pkl' % (nbyz, run)), 'wb'))

r 0 nmal 10 | e 0 benign_norm 1132.689 val loss 2.313 val acc 0.100 best val_acc 0.100
r 0 nmal 10 | e 5 benign_norm 276.867 val loss 2.207 val acc 0.185 best val_acc 0.185
r 0 nmal 10 | e 10 benign_norm 274.338 val loss 1.661 val acc 0.382 best val_acc 0.382
r 0 nmal 10 | e 15 benign_norm 273.673 val loss 1.694 val acc 0.384 best val_acc 0.384
r 0 nmal 10 | e 20 benign_norm 274.293 val loss 1.673 val acc 0.413 best val_acc 0.414
r 0 nmal 10 | e 25 benign_norm 275.130 val loss 1.481 val acc 0.473 best val_acc 0.473
r 0 nmal 10 | e 30 benign_norm 276.635 val loss 1.411 val acc 0.511 best val_acc 0.511
r 0 nmal 10 | e 35 benign_norm 278.580 val loss 1.329 val acc 0.539 best val_acc 0.561
r 0 nmal 10 | e 40 benign_norm 280.458 val loss 1.314 val acc 0.553 best val_acc 0.561
r 0 nmal 10 | e 45 benign_norm 284.015 val loss 1.302 val acc 0.563 best val_acc 0.574
r 0 nmal 10 | e 50 benign_norm 288.785 val loss 1.247 val acc 0.585 best val_acc 0.585
r 0 nmal 10 | e 55 benign_norm 294.493 val l

# With dirichlet distribution

In [13]:
all_data = torch.utils.data.ConcatDataset((cifar10_train, cifar10_test))
all_test_data = torch.utils.data.ConcatDataset((te_cifar10_train, te_cifar10_test))
batch_size = 16
num_workers = 100
distribution='dirichlet'
param = .1
force = False

each_worker_idx, each_worker_te_idx, global_test_idx = get_federated_data(
    all_data, num_workers=100, distribution=distribution, param=param, force=force)

train_loaders = []
for pos, indices in enumerate(each_worker_idx):
    train_loaders.append((pos, get_train(all_data, indices, batch_size)))
test_loaders = []
for pos, indices in enumerate(each_worker_te_idx):
    test_loaders.append((pos, get_train(all_test_data, indices, len(indices))))
cifar10_test_loader = get_train(all_test_data, global_test_idx)

len(train_loaders), len(test_loaders), len(global_test_idx)

(100, 100, 10043)

# CIFAR10 + ResNetBN + Dirichlet-0.1 baseline

In [15]:
'''
FAST BASELINE
'''

use_cuda = torch.cuda.is_available()
criterion = nn.CrossEntropyLoss()
torch.cuda.empty_cache()
nepochs=200
local_epochs = 2
batch_size = 16
num_workers = 100
local_lr = 0.1
global_lr = 1
agr = 'average'
best_global_acc=0
epoch_num = 0
nbyz = 0

fed_model = resnet20().cuda()
model_received = []
for i, (name, param) in enumerate(fed_model.state_dict().items()):
    model_received = param.view(-1).data.type(torch.cuda.FloatTensor) if len(model_received) == 0 else torch.cat((model_received, param.view(-1).data.type(torch.cuda.FloatTensor)))

while epoch_num <= nepochs:
    torch.cuda.empty_cache()
    round_clients = np.arange(20-nbyz, num_workers)
    round_benign = round_clients
    user_grads=[]
    for i in round_benign:
        model = copy.deepcopy(fed_model)
        optimizer = optim.SGD(model.parameters(), lr = local_lr*(0.999**epoch_num), momentum=0.9, weight_decay=1e-4)

        for epoch in range(local_epochs):
            train_loss, train_acc = train(train_loaders[i][1], model, model_received, criterion, optimizer)

        params = []
        for i, (name, param) in enumerate(model.state_dict().items()):
            params = param.view(-1).data.type(torch.cuda.FloatTensor) if len(params) == 0 else torch.cat((params, param.view(-1).data.type(torch.cuda.FloatTensor)))

        update =  (params - model_received)
        user_grads = update[None,:] if len(user_grads) == 0 else torch.cat((user_grads, update[None,:]), 0)

    agg_grads = torch.mean(user_grads, 0)
    del user_grads
    model_received = model_received + global_lr * agg_grads
    fed_model = resnet20().cuda()
    start_idx=0
    state_dict = {}
    previous_name = 'none'
    for i, (name, param) in enumerate(fed_model.state_dict().items()):
        start_idx = 0 if i == 0 else start_idx + len(fed_model.state_dict()[previous_name].data.view(-1))
        start_end = start_idx + len(fed_model.state_dict()[name].data.view(-1))
        params = model_received[start_idx:start_end].reshape(fed_model.state_dict()[name].data.shape)
        state_dict[name] = params
        previous_name = name
    fed_model.load_state_dict(state_dict)
    val_loss, val_acc = test(cifar10_test_loader, fed_model, criterion)
    is_best = best_global_acc < val_acc
    best_global_acc = max(best_global_acc, val_acc)
    if is_best:
        best_model = copy.deepcopy(fed_model)

    if epoch_num%10==0 or epoch_num==nepochs-1:
        print('e %d val loss %.3f val acc %.3f best val_acc %.3f'% (epoch_num, val_loss, val_acc, best_global_acc))
    if math.isnan(val_loss) or val_loss > 100000:
        print('val loss %f... exit'%val_loss)
        break
    epoch_num+=1

e 0 val loss 2.313 val acc 0.100 best val_acc 0.100
e 10 val loss 1.978 val acc 0.292 best val_acc 0.292
e 20 val loss 1.725 val acc 0.397 best val_acc 0.397
e 30 val loss 1.512 val acc 0.497 best val_acc 0.497
e 40 val loss 1.354 val acc 0.553 best val_acc 0.553
e 50 val loss 1.231 val acc 0.593 best val_acc 0.593
e 60 val loss 1.141 val acc 0.625 best val_acc 0.631
e 70 val loss 1.090 val acc 0.640 best val_acc 0.653
e 80 val loss 1.008 val acc 0.666 best val_acc 0.679
e 90 val loss 0.948 val acc 0.693 best val_acc 0.695
e 100 val loss 0.926 val acc 0.696 best val_acc 0.704
e 110 val loss 0.869 val acc 0.720 best val_acc 0.722
e 120 val loss 0.878 val acc 0.713 best val_acc 0.731
e 130 val loss 0.824 val acc 0.737 best val_acc 0.740
e 140 val loss 0.803 val acc 0.735 best val_acc 0.747
e 150 val loss 0.790 val acc 0.741 best val_acc 0.756
e 160 val loss 0.768 val acc 0.749 best val_acc 0.760
e 170 val loss 0.746 val acc 0.759 best val_acc 0.764
e 180 val loss 0.744 val acc 0.761 best

In [ ]:
'''
FAST BASELINE
'''

use_cuda = torch.cuda.is_available()
criterion = nn.CrossEntropyLoss()
torch.cuda.empty_cache()
nepochs=200
local_epochs = 2
batch_size = 8
train_loaders = []
for pos, indices in enumerate(each_worker_idx):
    train_loaders.append((pos, get_train(all_data, indices, batch_size)))
num_workers = 100
local_lr = 0.1
global_lr = 1
agr = 'average'
best_global_acc=0
epoch_num = 0
nbyz = 0

fed_model = resnet20().cuda()
model_received = []
for i, (name, param) in enumerate(fed_model.state_dict().items()):
    model_received = param.view(-1).data.type(torch.cuda.FloatTensor) if len(model_received) == 0 else torch.cat((model_received, param.view(-1).data.type(torch.cuda.FloatTensor)))

while epoch_num <= nepochs:
    torch.cuda.empty_cache()
    round_clients = np.arange(20-nbyz, num_workers)
    round_benign = round_clients
    user_grads=[]
    for i in round_benign:
        model = copy.deepcopy(fed_model)
        optimizer = optim.SGD(model.parameters(), lr = local_lr*(0.999**epoch_num), momentum=0.9, weight_decay=1e-4)

        for epoch in range(local_epochs):
            train_loss, train_acc = train(train_loaders[i][1], model, model_received, criterion, optimizer)

        params = []
        for i, (name, param) in enumerate(model.state_dict().items()):
            params = param.view(-1).data.type(torch.cuda.FloatTensor) if len(params) == 0 else torch.cat((params, param.view(-1).data.type(torch.cuda.FloatTensor)))

        update =  (params - model_received)
        user_grads = update[None,:] if len(user_grads) == 0 else torch.cat((user_grads, update[None,:]), 0)

    agg_grads = torch.mean(user_grads, 0)
    del user_grads
    model_received = model_received + global_lr * agg_grads
    fed_model = resnet20().cuda()
    start_idx=0
    state_dict = {}
    previous_name = 'none'
    for i, (name, param) in enumerate(fed_model.state_dict().items()):
        start_idx = 0 if i == 0 else start_idx + len(fed_model.state_dict()[previous_name].data.view(-1))
        start_end = start_idx + len(fed_model.state_dict()[name].data.view(-1))
        params = model_received[start_idx:start_end].reshape(fed_model.state_dict()[name].data.shape)
        state_dict[name] = params
        previous_name = name
    fed_model.load_state_dict(state_dict)
    val_loss, val_acc = test(cifar10_test_loader, fed_model, criterion)
    is_best = best_global_acc < val_acc
    best_global_acc = max(best_global_acc, val_acc)
    if is_best:
        best_model = copy.deepcopy(fed_model)

    if epoch_num%10==0 or epoch_num==nepochs-1:
        print('e %d val loss %.3f val acc %.3f best val_acc %.3f'% (epoch_num, val_loss, val_acc, best_global_acc))
    if math.isnan(val_loss) or val_loss > 100000:
        print('val loss %f... exit'%val_loss)
        break
    epoch_num+=1

e 0 val loss 2.306 val acc 0.099 best val_acc 0.099
e 10 val loss 1.989 val acc 0.246 best val_acc 0.246
e 20 val loss 1.635 val acc 0.415 best val_acc 0.415
e 30 val loss 1.424 val acc 0.504 best val_acc 0.504
e 40 val loss 1.290 val acc 0.553 best val_acc 0.553
e 50 val loss 1.188 val acc 0.595 best val_acc 0.595
e 60 val loss 1.094 val acc 0.632 best val_acc 0.632
e 70 val loss 1.025 val acc 0.656 best val_acc 0.656


In [ ]:
'''
FAST BASELINE
'''

use_cuda = torch.cuda.is_available()
criterion = nn.CrossEntropyLoss()
n_malicious = [10]
n_runs = 3
results={}
for n_mal in n_malicious:
    nbyz = n_mal
    results[n_mal] = {'acc': [], 'fpr': [], 'fnr': [], 'auc': [], 'stop_e': []}
    for run in range(n_runs):
        torch.cuda.empty_cache()
        nepochs=200
        local_epochs = 2
        batch_size = 16
        num_workers = 100
        local_lr = 0.1
        global_lr = 1
        agr = 'tr_mean'
        best_global_acc=0
        epoch_num = 0

        fed_model = resnet20().cuda()
        model_received = []
        for i, (name, param) in enumerate(fed_model.state_dict().items()):
            model_received = param.view(-1).data.type(torch.cuda.FloatTensor) if len(model_received) == 0 else torch.cat((model_received, param.view(-1).data.type(torch.cuda.FloatTensor)))

        best_accs_per_round = []
        accs_per_round = []
        loss_per_round = []
        home_dir = '/home/vshejwalkar_umass_edu/fedrecover/'

        # FLDetector initializations
        weight_record = []
        grad_record = []
        test_grads = []
        old_grad_list = []
        malicious_scores = np.zeros((1, num_workers-(20-nbyz)))
        start_detection_epoch = 10
        window_size = 10
        assert (start_detection_epoch - window_size >= 0), 'start_detection_epoch %d should be more than window_size %d' % (start_detection_epoch, window_size)

        # Adaptive attack initializers
        good_distance_range = np.zeros((1, nbyz))
        attack_type = 'NDSS21'
        dev_type = 'std'
        
        while epoch_num <= nepochs:
            torch.cuda.empty_cache()
            round_clients = np.arange(20-nbyz, num_workers)
            round_benign = round_clients
            user_grads=[]
            benign_norm = 0
            for i in round_benign:
                model = copy.deepcopy(fed_model)
                optimizer = optim.SGD(model.parameters(), lr = local_lr*(0.999**epoch_num), momentum=0.9, weight_decay=1e-4)

                for epoch in range(local_epochs):
                    train_loss, train_acc = train(train_loaders[i][1], model, model_received, criterion, optimizer)

                params = []
                for i, (name, param) in enumerate(model.state_dict().items()):
                    params = param.view(-1).data.type(torch.cuda.FloatTensor) if len(params) == 0 else torch.cat((params, param.view(-1).data.type(torch.cuda.FloatTensor)))

                update =  (params - model_received)
                benign_norm += torch.norm(update)/len(round_benign)
                user_grads = update[None,:] if len(user_grads) == 0 else torch.cat((user_grads, update[None,:]), 0)

            weight = model_received

            if (epoch_num > start_detection_epoch):
                hvp = lbfgs(weight_record, grad_record, weight - last_weight)
                hvp = np.squeeze(hvp)
            else:
                hvp = None

            good_current_grads = copy.deepcopy(user_grads[:nbyz])
            if hvp is not None:
                pred_grad = copy.deepcopy(good_old_grads)
                distance = []
                for i in range(len(good_old_grads)):
                    pred_grad[i] += torch.from_numpy(hvp).to(device)
                good_distance_range = np.concatenate(
                    (good_distance_range, torch.norm(pred_grad - good_current_grads, dim = 1).cpu().numpy()[None,:]), 0)

            if attack_type != 'none' and (epoch_num < start_detection_epoch):
                user_grads = full_trim(user_grads, nbyz)
                # user_grads[:nbyz] = full_trim(user_grads[:nbyz], nbyz)
            elif epoch_num > start_detection_epoch:
                if attack_type == 'full_trim':
                    user_grads = full_trim(user_grads, nbyz)
                elif attack_type == 'none':
                    pass
                else:
                    if attack_type == 'NDSS21':
                        distance_bound = np.mean(np.mean(good_distance_range[-1:], 0))
                        model_re = torch.mean(good_current_grads, dim=0)
                        if dev_type == 'unit_vec':
                            deviation = model_re / torch.norm(model_re)  # unit vector, dir opp to good dir
                        elif dev_type == 'sign':
                            deviation = torch.sign(model_re)
                        elif dev_type == 'std':
                            deviation = torch.std(good_current_grads, 0)
                        # noise = deviation * ((distance_bound + np.random.uniform(0, np.std(good_distance_range[-1]))) / torch.norm(deviation))
                        noise = deviation * ((distance_bound)) / torch.norm(deviation)
                    elif attack_type == 'mod_trim':
                        mal_grads= full_trim(user_grads[:nbyz], nbyz)
                        pass
                    else:
                        noise = torch.zeros(hvp.shape).to(device)
                    for m in range(nbyz):
                        user_grads[m] = old_grad_list[m] + torch.from_numpy(hvp).to(device) + noise

            agg_grads, distance = fldetector(old_grad_list, user_grads, nbyz, hvp, agr=agr)

            if distance is not None and epoch_num > (start_detection_epoch - window_size):
                malicious_scores = np.concatenate((malicious_scores, distance[None, :]), 0)

            if malicious_scores.shape[0] >= window_size+1:
                if detection1(np.sum(malicious_scores[-window_size:], axis=0), nbyz):
                    print('!!!! Stop at iteration:', epoch_num)
                    print('r %d nmal %d | e %d benign_norm %.3f val loss %.3f val acc %.3f best val_acc %.3f'% (run, nbyz, epoch_num, benign_norm, val_loss, val_acc, best_global_acc))
                    acc, fpr, fnr, auc = detection(
                        np.sum(malicious_scores[-window_size:], axis=0), nbyz, len(user_grads))
                    results[n_mal]['acc'].append(acc)
                    results[n_mal]['fpr'].append(fpr)
                    results[n_mal]['fnr'].append(fnr)
                    results[n_mal]['auc'].append(auc)
                    results[n_mal]['stop_e'].append(epoch_num)
                    break
                    break

            if epoch_num > (start_detection_epoch - window_size):
                weight_record.append(weight - last_weight)
                grad_record.append(agg_grads - last_grad)

            if (len(weight_record) > window_size):
                del weight_record[0]
                del grad_record[0]

            last_weight = weight
            last_grad = agg_grads
            old_grad_list = user_grads
            good_old_grads = good_current_grads
            del user_grads

            model_received = model_received + global_lr * agg_grads
            fed_model = resnet20().cuda()
            start_idx=0
            state_dict = {}
            previous_name = 'none'
            for i, (name, param) in enumerate(fed_model.state_dict().items()):
                start_idx = 0 if i == 0 else start_idx + len(fed_model.state_dict()[previous_name].data.view(-1))
                start_end = start_idx + len(fed_model.state_dict()[name].data.view(-1))
                params = model_received[start_idx:start_end].reshape(fed_model.state_dict()[name].data.shape)
                state_dict[name] = params
                previous_name = name
            fed_model.load_state_dict(state_dict)
            val_loss, val_acc = test(cifar10_test_loader, fed_model, criterion)
            is_best = best_global_acc < val_acc
            best_global_acc = max(best_global_acc, val_acc)
            if is_best:
                best_model = copy.deepcopy(fed_model)

            best_accs_per_round.append(best_global_acc.cpu().item())
            accs_per_round.append(val_acc.cpu().item())
            loss_per_round.append(val_loss.cpu().item())

            if epoch_num%10==0 or epoch_num==nepochs-1:
                print('r %d nmal %d | e %d benign_norm %.3f val loss %.3f val acc %.3f best val_acc %.3f'% (run, nbyz, epoch_num, benign_norm, val_loss, val_acc, best_global_acc))
            if math.isnan(val_loss) or val_loss > 100000:
                print('val loss %f... exit'%val_loss)
                break
            epoch_num+=1

        final_accs_per_client=[]
        for i in range(num_workers):
            client_loss, client_acc = test(test_loaders[i][1], best_model, criterion)
            final_accs_per_client.append(client_acc.cpu().item())
        all_results = collections.OrderedDict(
            final_accs_per_client=np.array(final_accs_per_client),
            accs_per_round=np.array(accs_per_round),
            best_accs_per_round=np.array(best_accs_per_round),
            loss_per_round=np.array(loss_per_round),
            results=results
        )
        pickle.dump(all_results, open(os.path.join(home_dir, 'FLDetector_plots_data/rq4_adaptive_cifar10_fast_trmean_NDSS21_std_m%d_r%d_dirichlet0.1.pkl' % (nbyz, run)), 'wb'))

r 0 nmal 10 | e 0 benign_norm 2390.534 val loss 2.315 val acc 0.099 best val_acc 0.099
r 0 nmal 10 | e 10 benign_norm 282.932 val loss 2.265 val acc 0.132 best val_acc 0.132
r 0 nmal 10 | e 20 benign_norm 280.659 val loss 1.850 val acc 0.330 best val_acc 0.330
r 0 nmal 10 | e 30 benign_norm 283.818 val loss 1.682 val acc 0.368 best val_acc 0.370
r 0 nmal 10 | e 40 benign_norm 288.321 val loss 1.553 val acc 0.419 best val_acc 0.420
